# Part 1: Getting the data

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [ ]:
# Get the file

wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

fp = urllib.request.urlopen(wiki_url)
mybytes = fp.read()
mystr = mybytes.decode("utf8")
fp.close()

In [ ]:
# Create the dataframe, give it proper columns

soup = BeautifulSoup(mystr)
table = str(soup.find_all("table", class_="wikitable")[0])
table
df = pd.read_html(table)[0]
df.columns = ['PostalCode', 'Borough', 'Neighborhood']

In [ ]:
# Remove bad borough values
#df = df.replace('Not assigned', pd.np.nan).dropna(axis=0, how='any', subset=['Borough'])
filter = df['Borough'] != 'Not assigned'
df.where(filter, inplace=True)
df.dropna(axis=0, inplace=True)

In [ ]:
# Show that we have a value where Neighbourhood is not assigned

df[df['Neighborhood'] == 'Not assigned']

In [ ]:
# Set neighbourhood = borough where neighbourhood = 'Not assigned'

mask = (df['Neighborhood'] == 'Not assighned')
df['Neighborhood'][mask]

In [ ]:
df[df['Neighborhood'] == 'Not assigned']

In [ ]:
df.head()

In [ ]:
df['Neighborhood'] = df.apply(
    lambda row: row['Borough'] if row['Neighborhood'] == 'Not assigned' else row['Neighborhood'], axis=1
)

In [ ]:
# Fixed the problem
df[df['Borough'] == 'Queen\'s Park']

In [ ]:
# Now to join places with the same postal code and borough

df = pd.DataFrame((df.groupby(['PostalCode','Borough'])['Neighborhood']
       .apply(lambda x: ','.join(set(x.dropna()))))).reset_index()
df 

In [ ]:
df.shape

# Section 2: Appending latitude and longitude

In [ ]:
# Import the CSV file

df_loc = pd.read_csv('Geospatial_Coordinates.csv')

In [ ]:
# Combine the two
df = pd.concat([df, df_loc], axis=1)


In [ ]:
df = df.drop(["Postal Code"], axis=1)
df

# Section 3: Foursquare and mapping

Take only boroughs that contain the word Toronto

In [ ]:
toronto_data = df[df.apply(lambda x: True if "Toronto" in x["Borough"] else False, axis=1)]
toronto_data.head()

In [ ]:
# Set up url parameters

CLIENT_ID = '0V3TLIC2XB0C0WRJW5YRZXJBQEUACTOWUWUNV2N3MZZQJCC1'
CLIENT_SECRET = 'ZLHT4HW1R4IQRPBOUR3GXLLVULWXPHMB20YESYRQZ2VITVV2'
VERSION = '20180605'
radius = 500
LIMIT = 100

In [ ]:
# Define the nearby venues function

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

**One hot encoding so we can analyze the venue data

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[159]] + list(toronto_onehot.columns[:159]) + list(toronto_onehot.columns[160:])

toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_onehot.shape

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

In [ ]:
toronto_grouped.shape

**Now it's time for some clustering!**

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=24).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

In [ ]:
toronto_grouped.insert(1, 'Cluster Label', kmeans.labels_)

In [ ]:
clustered_labels = toronto_grouped[['Neighborhood', 'Cluster Label']]

In [ ]:
toronto_merged = toronto_data.join(clustered_labels.set_index('Neighborhood'), on="Neighborhood")
toronto_merged.head()

**Let's visualize this clustering data and get out of here!**

In [ ]:
latitude = 43.6532
longitude = -79.3832

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

If there isn't a picture of the map, a picture of the map will be hosted in the github repository
https://github.com/yauneyz/Neighborhood-Analysis/blob/master/Toronto%20Clustering%20Map.png

# Final Analysis

It looks like the neighborhoods in Toronto are more homegenous than one would expect. There are a couple big outliers, but the rest of the neighborhoods are more similar than they are different. This is somewhat similart to the New York data, where one of the clusters was larger than the others, but here it is far more pronounded.

Interestingly enough, all of the different clusters are further from the beach. This is to be expected, becasue costal neighborhoods have a good chance of being fundamentally different than others.